In [ ]:
import numpy as np
import pandas as pd

SOURCE_PATH = "crawled_data/"
DEST_PATH = "anonomized_data/"

def anonomizePage(page, post_map):
    print(page)
    df = pd.read_csv(SOURCE_PATH + "%d-users.csv.gz" % page)
    df['fb_id-a'] = df[col].map({
        r['index']: i
        for i, r in df['fb_id'].value_counts().reset_index().iterrows()
    })
    df['post_id-a'] = df['post_id'].map(post_map)
    (df.dropna().sort_values(['fb_id-a']).to_csv(
        DEST_PATH + "%d.csv" % page,
        float_format="%d",
        columns=['post_id-a', 'fb_id-a', 'type'],
        header=['post', 'user', 'type'],
        index=False))

def anonomizeInteractions(interactions):
    post_map = interactions['post_id'].drop_duplicates().reset_index(drop=True)
    post_map = pd.Series(post_map.index.values, index=post_map)
    interactions['post_id'] = interactions['post_id'].map(post_map)
    interactions.to_csv(DEST_PATH + '00__combinedPageInteractions.csv',
                     float_format="%d", index=False)
    return post_map

In [ ]:
interactions = pd.read_csv(SOURCE_PATH + 'combinedPageInteractions.csv.bz2', parse_dates=[7])
post_map = anonomizeInteractions(interactions)
pages = interactions.page_id.unique()
print(len(pages))

for page in pages:
    print('Anonomizes {}'.format(page))
    anonomizePage(page, post_map)